In [1]:
import requests
IS_PROD = False
env = open("../.env.production" if IS_PROD else "../.env.development").readlines()
DISCORD_APPLICATION_ID = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith('DISCORD_APPLICATION_ID')][0]
DISCORD_BOT_TOKEN = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith('DISCORD_BOT_TOKEN')][0]
headers = {
    "Authorization": f"Bot {DISCORD_BOT_TOKEN}"
}
from typing import Optional
def get_command_id(name: str) -> Optional[str]:
    url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"
    r = requests.get(url, headers=headers)
    d = r.json()
    results = [x.get("id") for x in d if x.get("name") == name]
    return results[0] if results else None

# LIST COMMANDS

In [19]:
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"
r = requests.get(url, headers=headers)
r.json()

[{'id': '912067168210931783',
  'application_id': '912000855186751489',
  'version': '937655825864998973',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'starter',
  'description': 'Generate a conversation starter given some topics',
  'dm_permission': None,
  'options': [{'type': 3,
    'name': 'topics',
    'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'. (default: 'ice breaker')"},
   {'type': 3,
    'name': 'players',
    'description': "List of challenged players separated by ','. Example '@user1,@user2'"}]},
 {'id': '937655936204562453',
  'application_id': '912000855186751489',
  'version': '937655936204562454',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'about',
  'description': 'Give information about the bot',
  'dm_permission': None},
 {'id': '937656007562240060',
  'application_id': '912000855186751489',
  'version': '939257957747675196',
  'defa

# /starter

In [3]:
name = "starter"
d = {
    "name": name,
    "description": "Generate a conversation starter given some topics",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','."
            + " Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
            "type": 3,
            "required": False,
        },
        {
            "name": "players",
            "description": "List of challenged players separated by ','. Example '@user1,@user2'",
            "type": 3,
            "required": False,
        }
    ]
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '912067168210931783',
 'application_id': '912000855186751489',
 'version': '937655825864998973',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'starter',
 'name_localizations': None,
 'description': 'Generate a conversation starter given some topics',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'players',
   'name_localizations': None,
   'description': "List of challenged players separated by ','. Example '@user1,@user2'",
   'description_localizations': None}]}

# COMMAND DELETE

In [18]:
name = "conf"
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/{COMMAND_ID}"
r = requests.delete(url, headers=headers)
r.text

''

# /about

Langame
A game where you can play with your friends.
How to play
Send this command to a channel:
/starter topics:ice breaker,travel,whatever topic your like

You can also add players to the Langame by adding the following option:
/starter players:@user1,@user2,@user3

With love, by https://langa.me/. Augmented conversations.


In [5]:
name = "about"
d = {
    "name": name,
    "description": "Give information about the bot",
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937655936204562453',
 'application_id': '912000855186751489',
 'version': '937655936204562454',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'about',
 'name_localizations': None,
 'description': 'Give information about the bot',
 'description_localizations': None,
 'dm_permission': None}

# /sub

/sub topics:ice breaker,travel players:@louis030195,@bob27 frequency:12h

results in the bot posting conversation starters between these two every 12h on the given topics

Optionally use players:random to have the bot choose a random player from the server


### Execution

Topics:ice breaker,travel
Players:@louis030195,@bob27

What is your favorite place?

CTA component button to stop the sub, like button click on /unsub whatever

In [2]:
name = "sub"
d = {
    "name": name,
    "description": "Subscribe to frequent Langames",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','. Example 'ice breaker,travel,dating'"
            + "(default 'ice breaker')",
            "type": 3,
            "required": False,
        },
        # how the players are selected?
        {
            "name": "players",
            "description": "How are the players selected on this server? (default: 'Random on this server')",
            "type": 3,
            "required": False,
            "choices": [
                {
                    "name": "Random on this server",
                    "value": "random_server"
                },
                {
                    "name": "Most talkative people on this channel",
                    "value": "talkative_channel"
                },
                # {
                #     "name": "Random on this channel",
                #     "value": "random_channel"
                # }
            ]
        },
        {
            "name": "frequency",
            "description": "Frequency in minutes,"
            + " hours or days. Example '30m' or '5h' or '2d'"
            + " (default is 3h)",
            "type": 3,
            "required": False,
        }
    ],
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937656007562240060',
 'application_id': '912000855186751489',
 'version': '939257957747675196',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'sub',
 'name_localizations': None,
 'description': 'Subscribe to frequent Langames',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'(default 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'players',
   'name_localizations': None,
   'description': "How are the players selected on this server? (default: 'Random on this server')",
   'description_localizations': None,
   'choices': [{'name': 'Random on this server',
     'value': 'random_server',
     'name_localizations': None},
    {'name': 'Most talkative people on this channel',
     'value': 'talkative_channel',
     'name_localizations'

# /unsub

Unsubscribe the server from Langames.

In [6]:
name = "unsub"
d = {
    "name": name,
    "description": "Unsubscribe from frequent Langames",
    "options": [
        {
            "name": "self",
            "description": "Prevent Langame from mentioning you",
            "type": 2,
            "options": [
                {
                    "name": "channel",
                    "description": "Prevent Langame from mentioning you on this channel",
                    "type": 1
                },
                {
                    "name": "guild",
                    "description": "Prevent Langame from mentioning you on the whole server",
                    "type": 1
                }
            ]
        },
        {
            "name": "global",
            "description": "Unsubscribe from all Langames",
            "type": 2,
            "options": [
                {
                    "name": "channel",
                    "description": "Unsubscribe from all Langames on this channel",
                    "type": 1
                },
                {
                    "name": "guild",
                    "description": "Unsubscribe from all Langames on the whole server",
                    "type": 1
                }
            ]
        }
    ],
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937402122624962611',
 'application_id': '920612693864493066',
 'version': '979269246443991080',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'unsub',
 'name_localizations': None,
 'description': 'Unsubscribe from frequent Langames',
 'description_localizations': None,
 'dm_permission': True,
 'options': [{'type': 2,
   'name': 'self',
   'name_localizations': None,
   'description': 'Prevent Langame from mentioning you',
   'description_localizations': None,
   'options': [{'type': 1,
     'name': 'channel',
     'name_localizations': None,
     'description': 'Prevent Langame from mentioning you on this channel',
     'description_localizations': None},
    {'type': 1,
     'name': 'guild',
     'name_localizations': None,
     'description': 'Prevent Langame from mentioning you on the whole server',
     'description_localizations': None}]},
  {'type': 2,
   'name': 'global',
   'name_localizations': None,
   'description': 'Unsubscri

# /setup

Setup various configuration for Langame Discord bot

In [2]:
name = "setup"
d = {
    "name": name,
    "description": "Setup various configuration for Langame Discord bot",
    "options": [
        {
            "name": "loquacity",
            "description": "How talkative the bot should be? (default: 'Extrovert')",
            "type": 3,
            "required": False,
            "choices": [
                {
                    "name": "Shy",
                    "value": "0",
                },
                {
                    "name": "Extrovert",
                    "value": "1",
                },
                {
                    "name": "Talkative",
                    "value": "2",
                },
                {
                    "name": "Gossipy",
                    "value": "3",
                }
            ]
        },
        {
            "name": "channel",
            "description": "Specific channel configuration, will remove global configuration",
            "type": 3,
            "required": False,
        },
        {
            "name": "translation",
            "description": "Enable/disable translation globally or for this channel",
            "type": 3,
            "required": False,
            "choices": [
                {
                    "name": "English",
                    "value": "en",
                },
                {
                    "name": "Deutsch",
                    "value": "de",
                },
                {
                    "name": "Español",
                    "value": "es",
                },
                {
                    "name": "Français",
                    "value": "fr",
                },
                {
                    "name": "Italiano",
                    "value": "it",
                },
                {
                    "name": "日本語",
                    "value": "ja",
                },
                {
                    "name": "한국어",
                    "value": "ko",
                },
                {
                    "name": "Português",
                    "value": "pt",
                },
                {
                    "name": "Русский",
                    "value": "ru",
                },
                {
                    "name": "简体中文",
                    "value": "zh-CN",
                },
            ]
        },
        {
            "name": "save",
            "description": "Save answers to Langame conversation starters.",
            "type": 3,
            "required": False,
            "choices": [
                {
                    "name": "Yes",
                    "value": "True",
                },
                {
                    "name": "No",
                    "value": "False",
                },
            ],
        },
    ],
}
COMMAND_ID = get_command_id(name)
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '952751388839002153',
 'application_id': '912000855186751489',
 'version': '962210150112985109',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'setup',
 'name_localizations': None,
 'description': 'Setup various configuration for Langame Discord bot',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'loquacity',
   'name_localizations': None,
   'description': "How talkative the bot should be? (default: 'Extrovert')",
   'description_localizations': None,
   'choices': [{'name': 'Shy', 'value': '0', 'name_localizations': None},
    {'name': 'Extrovert', 'value': '1', 'name_localizations': None},
    {'name': 'Talkative', 'value': '2', 'name_localizations': None},
    {'name': 'Gossipy', 'value': '3', 'name_localizations': None}]},
  {'type': 3,
   'name': 'channel',
   'name_localizations': None,
   'description': 'Specific channel configuration, will remove global configuration',
   'descrip

# Other

In [11]:
# url = f"https://discord.com/api/v8/guilds/888438975646285834/members/267955006974590976"
# url = f"https://discord.com/api/v8/guilds/888438975646285834/roles"
url = f"https://discord.com/api/v8/guilds/888438975646285834/members?limit=1000"
r = requests.get(url, headers=headers)
r.json()

[{'roles': ['933033976828948520', '933032577319731292'],
  'nick': None,
  'avatar': None,
  'premium_since': None,
  'joined_at': '2022-01-18T16:18:21.437000+00:00',
  'is_pending': False,
  'pending': False,
  'communication_disabled_until': None,
  'user': {'id': '172002275412279296',
   'username': 'Tatsu',
   'avatar': '43fc1879641f6239d6e4038eaa7dabbc',
   'discriminator': '8792',
   'public_flags': 589824,
   'bot': True},
  'mute': False,
  'deaf': False},
 {'roles': ['916740688396292216', '935092248167739432'],
  'nick': None,
  'avatar': None,
  'premium_since': None,
  'joined_at': '2021-09-17T14:59:18.242000+00:00',
  'is_pending': False,
  'pending': False,
  'communication_disabled_until': None,
  'user': {'id': '267955006974590976',
   'username': 'louis030195',
   'avatar': '1a8d658e1267a1b0b36f93c151e4c733',
   'discriminator': '2462',
   'public_flags': 0},
  'mute': False,
  'deaf': False},
 {'roles': ['933033976828948520', '935092248167739432'],
  'nick': None,
  'a

# Analysis

In [2]:
from langame import LangameClient
c = LangameClient("../config.yaml")
si = c._firestore_client.collection("social_interactions").order_by("guild_id").get()

In [3]:
guilds = {}
for e in si:
    g_id = e.get("guild_id")
    if g_id not in guilds:
        guilds[g_id] = set()
    guilds[g_id].add(e.get("channel_id"))
# turn into list
for g_id in guilds:
    guilds[g_id] = list(guilds[g_id])
guilds

{'413438066984747026': ['738645672085159946'],
 '756897801790554113': ['756898363563049054'],
 '888438975646285834': ['888438976128638998',
  '939263257179484160',
  '939263480345821245'],
 '937357295199846400': ['937357295199846403', '939244198631182386']}

In [5]:
for g_id in guilds:
    url = f"https://discord.com/api/v8/guilds/{g_id}"
    r =  requests.get(url, headers=headers)
    d = r.json()
    print("\nid:" + d.get("id", "has no id"))
    print("name:" + d.get("name", "has no name"))
    description = d.get("description", None)
    if description:
        print("description:" + description)
    print("channels:" + ",".join(guilds[g_id]))


id:413438066984747026
name:PrismarineJS
description:Building bots and servers for minecraft. Network, AI, NodeJs, packaging
channels:738645672085159946

id:756897801790554113
name:Polcompball
channels:756898363563049054

id:888438975646285834
name:Langame
description:Augmented conversations @ https://langa.me
channels:888438976128638998,939263257179484160,939263480345821245

id:937357295199846400
name:test
channels:937357295199846403,939244198631182386


In [9]:
c_id = "738645672085159946"
url = f"https://discord.com/api/v8/channels/{c_id}"
r =  requests.get(url, headers=headers)
d = r.json()
d

{'id': '738645672085159946',
 'last_message_id': '939552025929220218',
 'type': 0,
 'name': 'bot',
 'position': 41,
 'parent_id': '743199109514657952',
 'topic': 'This is a bot room',
 'guild_id': '413438066984747026',
 'permission_overwrites': [{'id': '413438066984747026',
   'type': 0,
   'allow': '0',
   'deny': '16384'}],
 'nsfw': False,
 'rate_limit_per_user': 0}

In [10]:
url = f"https://discord.com/api/v8/channels/{c_id}/messages"
r =  requests.get(url, headers=headers)
all_messages = r.json()
all_messages

[{'id': '939552025929220218',
  'type': 0,
  'content': 'sorry',
  'channel_id': '738645672085159946',
  'author': {'id': '756219590027247698',
   'username': 'ILYAS『MR』',
   'avatar': '85823d0aad0309a5a289bafd628af3e9',
   'discriminator': '1617',
   'public_flags': 256},
  'attachments': [],
  'embeds': [],
  'mentions': [],
  'mention_roles': [],
  'pinned': False,
  'mention_everyone': False,
  'tts': False,
  'timestamp': '2022-02-05T16:04:19.014000+00:00',
  'edited_timestamp': None,
  'flags': 0,
  'components': []},
 {'id': '939551636081225818',
  'type': 0,
  'content': '<@!756219590027247698> the next . is a ban',
  'channel_id': '738645672085159946',
  'author': {'id': '326082865035083778',
   'username': 'rom1504',
   'avatar': '3bee148daf5b55834a1474a269862e5f',
   'discriminator': '5008',
   'public_flags': 0},
  'attachments': [],
  'embeds': [],
  'mentions': [{'id': '756219590027247698',
    'username': 'ILYAS『MR』',
    'avatar': '85823d0aad0309a5a289bafd628af3e9',
   

In [23]:
from random import sample, randint
from typing import List
def get_most_talkative_players_from_channel(channel_id) -> List[dict]:
    """
    Get the most talkative players from a Discord channel.
    :param channel_id: the channel id
    """
    url = f"https://discord.com/api/v8/channels/{channel_id}/messages?limit=100"
    # TODO: https://discord.com/developers/docs/resources/channel#get-channel-messages
    # impl filtering of dates
    r =  requests.get(url, headers=headers)
    if r.status_code != 200:
        raise ValueError(f"Failed to get the list of messages for channel {channel_id}. {r.text}")
    all_messages = r.json()
    # find most talkative people on this channel
    messages_per_user = {}
    for e in all_messages:
        # filter out bots (role "Bot") or ["user"]["bot"]=True
        if e.get("author", {}).get("bot", False):
            continue
        author_id = e.get("author").get("id")
        if author_id not in messages_per_user:
            messages_per_user[author_id] = []
        messages_per_user[author_id].append(e)
    sorted_by_nb_messages = sorted(messages_per_user.items(), key=lambda x: len(x[1]), reverse=True)
    # take top three distinct users
    top_talkers = []
    for e in sorted_by_nb_messages:
        if e[1][0].get("author").get("id") not in [j.get("id") for j in top_talkers]:
            top_talkers.append(e[1][0].get("author"))
    return top_talkers
most_talkative_players = get_most_talkative_players_from_channel(c_id)
# 10% of the most talkative players
# k = int(len(most_talkative_players) / 10) if len(most_talkative_players) > 3 else 3
# most_talkative_players = most_talkative_players[:k]
# print(most_talkative_players)
# sample from most talkative players
players = sample(
    most_talkative_players,
    k=randint(2, 3) if len(most_talkative_players) > 2 else len(most_talkative_players),
)
# turn players into a list of ids
# players = [player.get("id") for player in players]
# players
players

[{'id': '259381108611743744',
  'username': 'extremeheat',
  'avatar': '2866e397ad2a3388581d918839ae864d',
  'discriminator': '7024',
  'public_flags': 0},
 {'id': '526776599505403904',
  'username': 'nab138',
  'avatar': 'e9757a3dc460c9fec133ea9e7b50f1a2',
  'discriminator': '2035',
  'public_flags': 128},
 {'id': '225964614897238016',
  'username': 'Ic3Tank',
  'avatar': '7a352e0df335cca867a5a0da252a21d1',
  'discriminator': '3548',
  'public_flags': 0}]

In [46]:
sorted_by_nb_messages

[('912000855186751489',
  [{'id': '937718051342794792',
    'type': 20,
    'content': "Topics: backpacking.\n\n**what would you do with a year of your life where you could do all sorts of things (life stuff, not money stuff) that you haven't had yet - what would they be?**",
    'channel_id': '937357295199846403',
    'author': {'id': '912000855186751489',
     'username': 'Langame',
     'avatar': '71c05f3e97fc8b6ebf015aad8e67186d',
     'discriminator': '8825',
     'public_flags': 524288,
     'bot': True},
    'attachments': [],
    'embeds': [],
    'mentions': [],
    'mention_roles': [],
    'pinned': False,
    'mention_everyone': False,
    'tts': False,
    'timestamp': '2022-01-31T14:36:45.399000+00:00',
    'edited_timestamp': '2022-01-31T14:36:51.525000+00:00',
    'flags': 0,
    'components': [],
    'application_id': '912000855186751489',
    'interaction': {'id': '937718050147414026',
     'type': 2,
     'name': 'starter',
     'user': {'id': '267955006974590976',
  